In [ ]:
### 이 명령어를 통해 찾은 최적의 하이퍼파라미터 조합은 아래 !python run.py에서 활용되었습니다. run.py 파일 보시면 해당 하이퍼파라미터들이 적혀있습니다. 
!python hyperparameter.py --epochs=2

---
#### 4가지 실험 모델(위의 명령어를 통해서 찾은 최적의 하이퍼파라미터)을 돌리는 터미널 실행 명령어입니다. 

In [2]:
!python run.py

path logger: runs\run_final

2022-02-13 17:54:15.678283: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-02-13 17:54:15.679028: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



Val Loss: 2.2717, valid Accuracy: 38.54 ± nan, Duration: 25220.496 



---
#### runs 디렉터리에 있는 모든 실행결과 로그를 확인할 수 있는 (텐서보드로) 명령어입니다.  텐서보드 접속은 기본 default port 번호 사용해서 접속하면 됩니다

In [16]:
!tensorboard --logdir=runs

^C


---
#### 제공된 test data셋에 대해서 에측 및 결과 저장

In [7]:
import torch
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score
from custom_datasets import CustomDataset



def evaluate(model, data):
    model.eval()

    with torch.no_grad():
        logits = model(data)

    outs = {}
    mask = data.test_mask
    key ='test'             
        
    loss = F.nll_loss(logits[mask], data.y[mask]).item()
    pred = logits[mask].max(1)[1]
    acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()

    return pred


dataset = CustomDataset()
model = torch.load('save/model_fin.pth')
outs = evaluate(model, dataset)
print(outs)



tensor([24, 16, 16,  ..., 24,  4,  8])


In [12]:
import pandas as pd

result = pd.read_csv('data/node_label_test.csv')

result.columns = ['id', 'pred']
result.pred = outs
result.to_csv('result/result.csv', index = False)